# VacationPy

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,fremont,37.5483,-121.9886,33.15,56,0,6.17,US,1694560911
1,1,vila velha,-20.3297,-40.2925,22.97,89,20,4.12,BR,1694561688
2,2,sola,-13.8833,167.5500,25.09,82,79,8.20,VU,1694561688
3,3,belmonte,-15.8631,-38.8828,22.78,84,9,2.34,BR,1694561689
4,4,stanley,54.8680,-1.6985,9.32,92,20,2.57,GB,1694561689


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles="OSM",
    frame_width = 700,
    frame_height = 500
    
)

map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the city_data_df DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_df = city_data_df[
    (city_data_df["Max Temp"]< 27 ) & (city_data_df["Max Temp"] > 21)  
    & (city_data_df["Wind Speed"] < 4.5) 
    & (city_data_df["Cloudiness"] ==0)
    
    
]

# Drop any rows with null values
narrowed_df = narrowed_df.dropna()

# Display sample data
narrowed_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,ghat,24.9647,10.1728,26.77,24,0,1.56,LY,1694561692
51,51,bethel,41.3712,-73.4140,24.16,83,0,1.54,US,1694561658
156,156,westport,41.1415,-73.3579,24.86,80,0,2.06,US,1694561765
218,218,callaway,30.1530,-85.5699,26.65,82,0,3.60,US,1694561796
244,244,hutchinson,38.0608,-97.9298,24.30,43,0,1.54,US,1694561514


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_df[["City", "Country","Lat", "Lng", "Lng"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Lng,Hotel Name
10,ghat,LY,24.9647,10.1728,10.1728,
51,bethel,US,41.3712,-73.4140,-73.4140,
156,westport,US,41.1415,-73.3579,-73.3579,
218,callaway,US,30.1530,-85.5699,-85.5699,
244,hutchinson,US,38.0608,-97.9298,-97.9298,
262,rajaori,IN,33.3833,74.3000,74.3000,
280,babolsar,IR,36.7025,52.6576,52.6576,
292,battagram,PK,34.6772,73.0233,73.0233,
304,skiathos,GR,39.1623,23.4909,23.4909,
308,roswell,US,34.0232,-84.3616,-84.3616,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey" : geoapify_key,
    "limit": 20 
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row["Lat"]
    longitude = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ghat - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
westport - nearest hotel: No hotel found
callaway - nearest hotel: No hotel found
hutchinson - nearest hotel: No hotel found
rajaori - nearest hotel: No hotel found
babolsar - nearest hotel: No hotel found
battagram - nearest hotel: No hotel found
skiathos - nearest hotel: No hotel found
roswell - nearest hotel: No hotel found
klamath falls - nearest hotel: No hotel found
guaruja - nearest hotel: No hotel found
nerja - nearest hotel: No hotel found
chaman - nearest hotel: No hotel found
igaracu do tiete - nearest hotel: No hotel found
fort mcmurray - nearest hotel: No hotel found
pamandzi - nearest hotel: No hotel found
igoumenitsa - nearest hotel: No hotel found
san sebastian de la gomera - nearest hotel: No hotel found
maysville - nearest hotel: No hotel found
mizdah - nearest hotel: No hotel found


,City,Country,Lat,Lng,Lng,Hotel Name
10,ghat,LY,24.9647,10.1728,10.1728,No hotel found
51,bethel,US,41.3712,-73.4140,-73.4140,No hotel found
156,westport,US,41.1415,-73.3579,-73.3579,No hotel found
218,callaway,US,30.1530,-85.5699,-85.5699,No hotel found
244,hutchinson,US,38.0608,-97.9298,-97.9298,No hotel found
262,rajaori,IN,33.3833,74.3000,74.3000,No hotel found
280,babolsar,IR,36.7025,52.6576,52.6576,No hotel found
292,battagram,PK,34.6772,73.0233,73.0233,No hotel found
304,skiathos,GR,39.1623,23.4909,23.4909,No hotel found
308,roswell,US,34.0232,-84.3616,-84.3616,No hotel found


In [28]:
name_address

{'statusCode': 400,
 'error': 'Bad Request',
 'message': '"filter[0]" does not match any of the allowed types'}

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [27]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles="OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel", "Country"]
    
)

map_plot

# Display the map


DataError: Dimensions may not reference duplicated DataFrame columns (found duplicate 'Lng' columns). If you want to plot a column against itself simply declare two dimensions with the same name. 

PandasInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html